data: willhaben: apartments for sale in vienna (no other filters)

In [33]:
from pathlib import Path
import pandas as pd
import numpy as np


files = (Path.cwd() / "data").iterdir()
files = sorted(files, key=lambda x: x.name)
file = files[0]
print(f"reading {file}")

df = pd.read_csv(file, sep=";")

print(len(df))
df.head()

reading /Users/sueszli/dev/will-wohnen/data/links_2024-05-24_19-19-36.csv
3150


,link,title,subheader,price,size,num_rooms,type
0,https://www.willhaben.at/iad/immobilien/d/eige...,PROVISIONSFREI! ERSTBEZUG! 29. STOCK! HELL! RU...,NaN,€ 379.800,50 m²,2 Zimmer,"Balkon, Loggia"
1,https://www.willhaben.at/iad/immobilien/d/eige...,Wohnung in absoluter Ruhelage mit Grünblick un...,NaN,"€ 299.000,25",50 m²,2 Zimmer,"Balkon, Loggia"
2,https://www.willhaben.at/iad/immobilien/d/eige...,Perfekte Starterwohnung in historischem Altbau...,NaN,"€ 350.000,44",65 m²,2 Zimmer,NaN
3,https://www.willhaben.at/iad/immobilien/d/eige...,Ruhepol in Aspern - Moderne 2-Zimmer Wohnung m...,NaN,"€ 425.000,25",56 m²,2 Zimmer,"Terrasse, Garten"
4,https://www.willhaben.at/iad/immobilien/d/eige...,Unbefristet vermietete Wohnung bei der U4-Frie...,NaN,"€ 120.000,71",26 m²,1 Zimmer,NaN


In [25]:
df = df.copy()




2610